# Live 15 - MLOps

In [1]:
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np
import pickle
import os
import sqlite3

In [2]:
os.chdir('C:\\Users\\dalto\\OpM na Prática\\Python\\Base de Dados')

mMod = pickle.load(open("IF.sav", "rb"))

In [3]:
def conexao(db):
    global conex
    try:
        conex = sqlite3.connect(db)
        print("Conexão realizada")
    except Error as e:
        print(e)

In [4]:
conexao(r"C:\Users\dalto\Desktop\DataScience\miniIndice.db")

Conexão realizada


In [5]:
df = pd.read_csv("mini_indice_D1.csv", sep = ";").to_sql(name = "mini", con = conex, index = False)

In [8]:
ponteiro = conex.cursor()
ponteiro.execute("""INSERT INTO mini (Data, Abertura, Maxima, Minima, Fechamento) 
                VALUES ("18/07/2020", 103350, 1050003, 102250, 104000)""")
conex.commit()

In [9]:
def cria_variaveis(df):
    df1 = df
    df1["Data"] = pd.to_datetime(df1["Data"]).dt.normalize()
    df1 = df1.set_index("Data")
    periodos = 1
    # lag positivo de X periodos
    df1["Lag_P1"] = df1["Fechamento"].shift(periodos)
    # Calculando o Retorno
    df1["Retorno"] = df1["Fechamento"].pct_change(periodos)
    # Desvio Padrao de 5 dias
    df1["std5"] = df1["Retorno"].rolling(5).std()
    # Desvio Padrao de 10 dias
    df1["std10"] = df1["Retorno"].rolling(10).std()
    # Desvio Padrao de 15 dias
    df1["std15"] = df1["Retorno"].rolling(15).std()
    # Proporçao do corpo do candle em relacao ao range do dia
    df1["prop"] = (df1["Fechamento"]-df1["Abertura"])/(df1["Maxima"]-df1["Minima"])
    # Direçao do dia atual
    df1["dir_D"] = np.where(df1['Fechamento'] > df1['Abertura'] , '1', '0')
    # Direçao D-1
    df1["dir_D-1"] = df1["dir_D"].shift(1)
    # Direçao D-2
    df1["dir_D-2"] = df1["dir_D"].shift(2)
    # Direçao D-3
    df1["dir_D-3"] = df1["dir_D"].shift(3)
    # Media Movel de 15 dias std5
    df1["mm_std5"] = df1["std5"].rolling(15).mean()
    # Media Movel de 15 dias std5
    df1["mm_std10"] = df1["std10"].rolling(15).mean()
    # Media Movel de 15 dias std5
    df1["mm_std15"] = df1["std15"].rolling(15).mean()
    # RSL std5
    df1["RSL_std5"] = (df1["std5"]/df1["std5"].rolling(15).mean())-1
    # RSL std10
    df1["RSL_std10"] = (df1["std10"]/df1["std10"].rolling(15).mean())-1
    # RSL std15
    df1["RSL_std15"] = (df1["std15"]/df1["std15"].rolling(15).mean())-1
    # Filtrando os valores missing
    df1_filtrado = df1.dropna(axis = 0) 
    df1_filtrado= df1_filtrado.drop(["std5","std10","std15","mm_std5","mm_std10","mm_std15"], axis = 1)
    df1_filtrado["RSL_std5"] = pd.qcut(df1_filtrado["RSL_std5"], 10, labels = False)
    df1_filtrado["RSL_std10"] = pd.qcut(df1_filtrado["RSL_std10"], 10, labels = False)
    df1_filtrado["RSL_std15"] = pd.qcut(df1_filtrado["RSL_std15"], 10, labels = False)
    return(df1_filtrado)

In [10]:
base = cria_variaveis(
    pd.read_sql(
        "select Data, Abertura, Maxima, Minima, Fechamento from mini"
        , conex)
)

In [11]:
base.head()

,Abertura,Maxima,Minima,Fechamento,Lag_P1,Retorno,prop,dir_D,dir_D-1,dir_D-2,dir_D-3,RSL_std5,RSL_std10,RSL_std15
Data,,,,,,,,,,,,,,
2005-06-28,74836.91,74981.11,73971.75,74115.94,74029.42,0.001169,-0.714284,0,1,0,0,6,2,0
2005-06-29,74404.33,74981.11,73683.36,73712.20,74115.94,-0.005447,-0.533331,0,0,1,0,6,1,0
2005-06-30,73827.55,74404.33,73539.16,73683.36,73712.20,-0.000391,-0.166661,0,0,0,1,0,1,0
2005-01-07,74115.94,74260.14,73885.23,74260.14,73683.36,0.007828,0.384626,1,0,0,0,0,1,0
2005-04-07,73250.77,73712.20,73106.58,73308.45,74260.14,-0.012816,0.095241,1,1,0,0,2,1,1


In [14]:
def predicao (tab):
    aux = mMod.predict(tab.iloc[:, 6:13])
    aux = np.where(aux == -1, 1, 0)
    mensagem = np.where(aux[-1] == 1, "Compra", "Vende")
    return (print("Previsao para amanha: " + str(mensagem)))

In [15]:
predicao(base)

Previsao para amanha: Compra


In [18]:
def predicao2():
    df1 = pd.read_sql(
        "select Data, Abertura, Maxima, Minima, Fechamento from mini"
        , conex)
    df1["Data"] = pd.to_datetime(df1["Data"]).dt.normalize()
    df1 = df1.set_index("Data")
    periodos = 1
    # lag positivo de X periodos
    df1["Lag_P1"] = df1["Fechamento"].shift(periodos)
    # Calculando o Retorno
    df1["Retorno"] = df1["Fechamento"].pct_change(periodos)
    # Desvio Padrao de 5 dias
    df1["std5"] = df1["Retorno"].rolling(5).std()
    # Desvio Padrao de 10 dias
    df1["std10"] = df1["Retorno"].rolling(10).std()
    # Desvio Padrao de 15 dias
    df1["std15"] = df1["Retorno"].rolling(15).std()
    # Proporçao do corpo do candle em relacao ao range do dia
    df1["prop"] = (df1["Fechamento"]-df1["Abertura"])/(df1["Maxima"]-df1["Minima"])
    # Direçao do dia atual
    df1["dir_D"] = np.where(df1['Fechamento'] > df1['Abertura'] , '1', '0')
    # Direçao D-1
    df1["dir_D-1"] = df1["dir_D"].shift(1)
    # Direçao D-2
    df1["dir_D-2"] = df1["dir_D"].shift(2)
    # Direçao D-3
    df1["dir_D-3"] = df1["dir_D"].shift(3)
    # Media Movel de 15 dias std5
    df1["mm_std5"] = df1["std5"].rolling(15).mean()
    # Media Movel de 15 dias std5
    df1["mm_std10"] = df1["std10"].rolling(15).mean()
    # Media Movel de 15 dias std5
    df1["mm_std15"] = df1["std15"].rolling(15).mean()
    # RSL std5
    df1["RSL_std5"] = (df1["std5"]/df1["std5"].rolling(15).mean())-1
    # RSL std10
    df1["RSL_std10"] = (df1["std10"]/df1["std10"].rolling(15).mean())-1
    # RSL std15
    df1["RSL_std15"] = (df1["std15"]/df1["std15"].rolling(15).mean())-1
    # Filtrando os valores missing
    df1_filtrado = df1.dropna(axis = 0) 
    df1_filtrado= df1_filtrado.drop(["std5","std10","std15","mm_std5","mm_std10","mm_std15"], axis = 1)
    df1_filtrado["RSL_std5"] = pd.qcut(df1_filtrado["RSL_std5"], 10, labels = False)
    df1_filtrado["RSL_std10"] = pd.qcut(df1_filtrado["RSL_std10"], 10, labels = False)
    df1_filtrado["RSL_std15"] = pd.qcut(df1_filtrado["RSL_std15"], 10, labels = False)
    # Faz a prediçao do modelo
    pred = mMod.predict(df1_filtrado.iloc[:, 6:13])
    pred = np.where(pred == -1, 1, 0)
    # Retorna a indicaçao
    mensagem = np.where(pred[-1] == 1, "Compra", "Vende")
    return (print("Previsao para amanha: " + str(mensagem)))

In [19]:
predicao2()

Previsao para amanha: Compra


In [20]:
conex.close()